<a href="https://colab.research.google.com/github/caumannerman/ncm_project/blob/master/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EA%B3%BC%EC%A0%9C_%EB%A7%88%EB%B2%95%EA%B3%B5%EC%8B%9D_%EC%98%A4%ED%9B%843%EC%8B%9C56%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pykrx import stock
import pandas as pd
import requests
import time
from datetime import datetime

In [4]:
# EV/EBTIDA 가져오기
# date='211230'
# year='2021/12'
def get_EVEBITDA(code, year):
  # print ("in get_EVEBITDA")
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A' + code + '&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

  fr_page = requests.get(url)
  fr_tables = pd.read_html(fr_page.text)

  temp_df = fr_tables[3]
  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['EV/EBITDA계산에 참여한 계정 펼치기']]
  temp_df.index = ['EV/EBITDA']
  temp_df = pd.DataFrame({code:temp_df[year]})
  temp_df = temp_df.transpose()
  # print (temp_df)
  return temp_df

In [5]:
# ROIC 가져오기
# date='211230'
# year='2021/12'
def get_ROIC(code, year):
  # print ("in get_ROIC")
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A'+ code +'&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

  fr_page = requests.get(url)
  fr_tables = pd.read_html(fr_page.text)

  temp_df = fr_tables[0]
  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['ROIC계산에 참여한 계정 펼치기']]
  temp_df.index = ['ROIC']
  temp_df = pd.DataFrame({code:temp_df[year]})
  temp_df = temp_df.transpose()
  print (temp_df)
  return temp_df

# print (get_ROIC('095570',year))

In [6]:
# 종목코드, 종목명 가져오기
# 2022년 1년간의 종목 수익률을 파악해야 하므로 종목 추출 시점은 2021년 연말이 되어야 함
# 따라서 종목, EV/EBTIDA, ROIC 모두 2021년 4분기 기준값으로 가져오기
date='211230'
year='2021/12'
codes = stock.get_market_ticker_list(date, market='ALL')
corp = []

for code in codes:
  name = stock.get_market_ticker_name(code)
  corp.append([code, name])

df = pd.DataFrame(data=corp, columns=['종목코드', '종목명'])
df_code = df['종목코드']

# EV/EBITDA dataframe 생성 및 적재
# 첫번재 종목이 3월 결산 종목이라 2021년 12월 데이터 값이 홈페이지에 나와있지 않음 -> 삼성전자를 임의로 첫번째 값으로 입력
evebitda_df=get_EVEBITDA('005930',year)

for num, code in enumerate(df_code):  
  # if num==10:
  #     break
  try:
      if num==1:
        return_df=get_EVEBITDA(code,year)
        evebitda_df=pd.concat([evebitda_df, return_df])
        print (evebitda_df)
        evebitda_df=evebitda_df.drop(index='005930')  
      else:
        return_df=get_EVEBITDA(code,year)
        evebitda_df=pd.concat([evebitda_df, return_df])
  except:
      pass
print (evebitda_df)

# ROIC dataframe 생성 및 적재
roic_df=get_ROIC('005930',year)

for num, code in enumerate(df_code):  
  # if num==10:
  #     break
  try:
      if num==1:
        return_df=get_ROIC(code,year)
        roic_df=pd.concat([roic_df, return_df])
        print (roic_df)
        roic_df=roic_df.drop(index='005930')  
      else:
        return_df=get_ROIC(code,year)
        roic_df=pd.concat([roic_df, return_df])
  except:
      pass
print (roic_df)


       EV/EBITDA
005930      6.23
095570      4.85
       EV/EBITDA
095570      4.85
006840     34.61
054620    274.52
265520      5.27
211270    139.60
...          ...
010240      5.49
189980     12.81
003280     72.73
037440      7.57
238490       NaN

[2366 rows x 1 columns]
        ROIC
005930  23.9
       ROIC
095570  2.0
        ROIC
005930  23.9
095570   2.0
        ROIC
006840  -5.2
        ROIC
054620  -8.1
        ROIC
265520  37.9
       ROIC
211270  1.9
       ROIC
027410  4.2
        ROIC
282330  18.2
       ROIC
001460  NaN
         ROIC
013720  -19.7
       ROIC
001040  4.7
        ROIC
079160  -7.3
       ROIC
035760  9.8
         ROIC
311690  -78.9
       ROIC
000120  3.9
       ROIC
011150  6.5
       ROIC
097950  6.4
        ROIC
051500  10.8
       ROIC
058820  2.2
        ROIC
056730  -1.4
       ROIC
065770  3.9
        ROIC
083660  25.0
        ROIC
000590  17.6
        ROIC
012030  31.1
        ROIC
000990  39.9
       ROIC
001530  9.0
       ROIC
000210  NaN
 

In [7]:
magic = pd.concat([evebitda_df,roic_df],axis=1)
print(magic)

       EV/EBITDA   ROIC
095570      4.85    2.0
006840     34.61   -5.2
054620    274.52   -8.1
265520      5.27   37.9
211270    139.60    1.9
...          ...    ...
037440      7.57   10.2
238490       NaN   -2.8
119650       NaN   40.6
044180       NaN  -40.1
123700       NaN    3.3

[2369 rows x 2 columns]


In [10]:
magic['evebitda_rank'] = magic['evebitda'].rank()
magic['roic_rank'] = magic['roic'].rank(ascending=False)
magic['total'] = magic['evebitda'] + magic['roic_rank']
magic['total_rank'] = magic['total'].rank()
magic = magic.sort_values(by=['total_rank'])
 
today = datetime.today().strftime('%y-%m-%d')
magic.to_excel('Magic(' + 'today' + ').xlsx')
 
return magic

print(make_magic_df(year, date))

KeyError: ignored